# <span style="color:darkblue"> Lecture 12 - Replacing and recoding variables</span>

<font size = "5">

- Starting a new module on **manipulating data**
- We will discuss NaNs and how to clean data

**Covered so far**

1. Python types and controlling flows (if/elif/else)

2. Local vs global variables

3. Writing and running functions and lambda functions and the apply method

4. Subsetting data using queries

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [2]:
import numpy as np
import pandas as pd

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [3]:
circuits = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

The dataset "codebook" is a table with ...

- Key column information
- Main things:  Field, Type, and Description

<img src="figures/codebook_circuits.png" alt="drawing" width="500"/>


In [4]:
# The codebook contains basics about the columns

# "Field" is the name given to the name of the column

# "Type"  is the variable type (with the number of characters in parenthesis):
#         integer (int)
#         string (varchar - "variable character") 
#         float (float)

# "Description" contains a label with the content of the variable

<font size = "5">

Quick Discussion:
- What does the "alt" column represent?
- What does "varchar(255)" mean?

# <span style="color:darkblue"> II. NaNs </span>

<font size = "5">

- Means "Not a Number"
- Used to denote missing values

In [5]:
# "NaNs" are a special number, available in the "np" library

np.nan

nan

<font size = "5">

Operations on arrays with NaNs

In [6]:
# Create two array with and without "NaNs"
# The "np.array()" functions converts a list to an array

vec_without_nans = np.array([1,1,1])
vec_with_nans = np.array([np.nan,4,5])

# When you add the vectors
# you will produce a NaN on any entries with "NaNs"
print(vec_without_nans * vec_with_nans)
print(vec_without_nans / vec_with_nans)
print(vec_without_nans + vec_with_nans)
print(vec_without_nans - vec_with_nans)

[nan  4.  5.]
[ nan 0.25 0.2 ]
[nan  5.  6.]
[nan -3. -4.]


<font size = "5">

Summary statistics with NaNs

In [7]:
# Some summary statistics will not work if there are "NaNs"
# The "np.mean()" doesn't work if the mean includes "NaNs"
print(np.mean(vec_with_nans))

# Some commands ignore the "nans"
# The "np.nanmean()" computes the mean over the numeric obvservations
print(np.nanmean(vec_with_nans))

nan
4.5


<font size = "5">

Pandas summary statistics ignore NaNs

In [8]:
# This command creates an empty dataframe
# then creates a new column called "x" and computes its mean
# Note: If a column contains missing values, then the "mean" won't be 
#       representative of the whole sample

dataset = pd.DataFrame()
dataset["x"] = vec_with_nans
dataset

,x
0,NaN
1,4.0
2,5.0


In [9]:
dataset["x"].mean()

4.5

# <span style="color:darkblue"> II. Data Cleaning</span>

<font size = "5">

- Data collection isn't perfect!
- Need to adjust values with incorrect formatting

<font size = "5">
Get data types

In [10]:
# Produces a list with the data types of each column
# Columns that say "object" have either strings or 
# a mix of string and numeric values

circuits.dtypes

circuitId       int64
circuitRef     object
name           object
location       object
country        object
lat           float64
lng           float64
alt            object
url            object
dtype: object

<font size = "5">

Check rows with numeric values

In [11]:
# The ".str.isnumeric()" checks whether each row is numeric or now.
# Using the "." twice is an example of "piping"
# which refers to chaining two commands "str" and "isnumeric()"

circuits["alt"].str.isnumeric()

0      True
1      True
2      True
3      True
4      True
      ...  
72     True
73     True
74     True
75    False
76    False
Name: alt, Length: 77, dtype: bool

In [12]:
circuits['location'].str.lower()

0        melbourne
1     kuala lumpur
2           sakhir
3         montmeló
4         istanbul
          ...     
72        portimão
73         mugello
74          jeddah
75       al daayen
76           miami
Name: location, Length: 77, dtype: object

<font size = "5">

Extract list of non-numeric values

In [16]:
# We can reference subattributes of columns in ".query()"
# The pd.unique() function extracts unique values from a list

subset = circuits.query("alt.str.isnumeric() == False")
list_unique = pd.unique(subset["alt"])
print(list_unique)

['\\N' '-7']


<font size = "5">

Replace certain values

In [30]:
# "list_old" encodes values we want to change
# "list_new" encodes the values that will "replace" the old
list_old = ['\\N','-7']
list_new = [np.nan, -7]

# This command replaces the values of the "alt" column
circuits["alt"] = circuits["alt"].replace(list_old, list_new)

<font size = "5">

Store a "cleaned" dataset

In [31]:
circuits.dtypes

circuitId         int64
circuitRef       object
name             object
location         object
country          object
lat             float64
lng             float64
alt              object
url              object
alt_numeric     float64
bins_alt       category
hemisphere     category
dtype: object

In [43]:
# After the cleaning process is done, you may want to store the dataset again
# It's recommended to do this in a separate file from the original
# That way you can always go back to the original if you made a coding error

circuits.to_csv("data_clean/circuits.csv")

<font size = "5">
Try it yourself!

- Use ".replace()" with the "country" column
- Replace "UK" with "United Kingdom"

In [44]:
# Write your own code
circuits["location"] = circuits["location"].replace("UK", "united kingdom")

<font size = "5">
Try it yourself!

- What is the column type of "lat" or "lng"?
- Does it have any string variables?
- Can we use ```str.isnumeric()``` here?

In [45]:
# Write your own code
circuits.dtypes

circuitId         int64
circuitRef       object
name             object
location         object
country          object
lat             float64
lng             float64
alt              object
url              object
alt_numeric     float64
bins_alt       category
hemisphere     category
dtype: object

<font size = "5">

# <span style="color:darkblue"> II. Recoding Numeric Variables </span>



<font size = "5">

<span style="color:red"> Controlled Pitfall </span> Computing a mean for a non-numeric column

In [46]:
circuits["alt"].dtype

dtype('O')

<font size = "5">

Convert column to numeric

In [47]:
# pd.to_numeric() converts a column to numeric
# Before you use this option, make sure to "clean" the variable
# as we did before by checking what the non-numeric values are

circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])
print(circuits["alt_numeric"].mean())

248.1891891891892


In [48]:
circuits["alt_numeric"].min()

-7.0

In [49]:
circuits["alt_numeric"].max()

2227.0

<font size = "5">

Recode values based on an interval <br>

$ \qquad x_{bin} = \begin{cases} ``A" &\text{ if \quad} x_1 < x \le x_2 \\
                                  ``B" &\text{ if \quad} x_2 < x \le x_3 \end{cases} $



In [50]:
bins_x = [0,2500, 5000]
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]

circuits["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

# Note: if we set bins_x = [float("-inf"),2500, float("inf")]
#       then intervals are "Less than or equal to 2500" and "Above 2500"
#       float("inf") and float("-inf") represent infinity and negative infinity
#       The "right" command indicates that the right interval is
#       "less than or equal to" or just "less than"

In [57]:
c2 = circuits[circuits['alt_numeric']>10].copy()

In [58]:
c2.loc[c2['circuitId'] == 4, 'name'] = 'other name'
c2

,circuitId,circuitRef,name,location,country,lat,lng,alt,url,alt_numeric,bins_alt,hemisphere
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...,18.0,Between 0 and 2500,north
3,4,catalunya,other name,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,109.0,Between 0 and 2500,north
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park,130.0,Between 0 and 2500,north
6,7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.50000,-73.52280,13,http://en.wikipedia.org/wiki/Circuit_Gilles_Vi...,13.0,Between 0 and 2500,north
7,8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.86420,3.16361,228,http://en.wikipedia.org/wiki/Circuit_de_Nevers...,228.0,Between 0 and 2500,north
...,...,...,...,...,...,...,...,...,...,...,...,...
68,69,americas,Circuit of the Americas,Austin,USA,30.13280,-97.64110,161,http://en.wikipedia.org/wiki/Circuit_of_the_Am...,161.0,Between 0 and 2500,north
69,70,red_bull_ring,Red Bull Ring,Spielberg,Austria,47.21970,14.76470,678,http://en.wikipedia.org/wiki/Red_Bull_Ring,678.0,Between 0 and 2500,north
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...,108.0,Between 0 and 2500,north
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit,255.0,Between 0 and 2500,north


<font size = "5">
Try it yourself!

- Create a new variable "hemisphere"
- Encode lattitude in (-90 and 0] as "south"
- Encode lattitude in (0 and 90] as "north"

In [60]:
# Write your own code
bins_x = [-90,0,90]
labels_x = ["south",
            "north"]

circuits["hemisphere"] = pd.cut(circuits["lat"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

circuits

,circuitId,circuitRef,name,location,country,lat,lng,alt,url,alt_numeric,bins_alt,hemisphere
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,10.0,Between 0 and 2500,south
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...,18.0,Between 0 and 2500,north
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...,7.0,Between 0 and 2500,north
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,109.0,Between 0 and 2500,north
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park,130.0,Between 0 and 2500,north
...,...,...,...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...,108.0,Between 0 and 2500,north
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit,255.0,Between 0 and 2500,north
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...,15.0,Between 0 and 2500,north
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,NaN,http://en.wikipedia.org/wiki/Losail_Internatio...,NaN,NaN,north
